## Tutorial 03: Recovering the force-field parameters of mW water model using force-matching method. 

* The mW (monoatomic water) is a coarse-grained water model dervied from Stillinger-Weber potential form [1-2]. The model quantitatively predicts many physical properties of water with the accuracy comparable to those of atomistic water models, while being much more computationally efficient.   

* Here we demonstrate how to use the force-matching techinques to recover the standard force-field parameters of mW water. Force-matching is a commonly used method to parameterize a coarse-grained model. The optimal parameters are generated by minimizing the mean squared differences between predicted forces and reference forces. [3,4]. Complex force field functional forms can be often dealt with, since sufficient intermolecular forces data are available.

### Step 1: generating the reference data

#### Generating the intermolecular forces and energy data. 

* A cubic box of 512 particles interacting via standard mW potential. 
* MD simulation at T=300K and P=1bar. 
* The configurations are generated every 5 ps and total of 500 configuratinos are generated. 
* The forces on every particle and potential energy per configuration are used as reference data. 
* The dumped coordinates and forces in the sampled configuration are in LAMMPS trajectory file format

* mW potential forms: 
<img src="sw_potential.png" width="600">

### Step 2: prepare the template LAMMPS input script 

#### Force matching can be easily done using the LAMMPS's "rerun" command

* The number of particles and box information are specified to proceed. These parameters will be updated once the "rerun" is used to read trajectory.
* The "rerun" will read the coordinates and box information per configuration and compute the forces

### Step 3: modify the optimization input file

* modfiy the weight, number of cores for running LAMMPS, number of cores for computing objective functions
* modify the LAMMPS command depending on the cluster's environment


#### units:  

real   

#### objective function defined: 
#### format is:  
#### 'matching type', 'sub_folder name',"weight of current objective functions", "cores for running sampling","cores for evaluating objective functions"
#### The corresponding python data type is: "string","string", "float","integer","integer"
#### Example: 

force mW_300K_1bar_500 1 2 2 bf 5000 w 0 1   

#### shell command to launch sampling in each folder :  
#### Python subprocess module launch these commands in non-blocking manner in each desired folder:
#### sampling input file names will be subsistuted as: in.force, in.rdf, in.isobar ... 
#### Default sampling package is LAMMPS 
#### Example:

module load intel/psxe-2019-64-bit && srun -n %d -N1 -c1 --exclusive --mpi=pmi2 lmp_ml_water < in.%s

#### output frequency: 
#### first column is for dumping current best parameters/obj 
#### second column is for dumping restart simplex 
#### Example:

5 5 

#### format is:
#### "force-field type name", "parameter1", "parameter2","parameter2", ...  
#### Example:

#### standard mW: 6.189  2.3925  1.80  23.15  1.20  -0.333333333333 7.049556277  0.6022245584  4.0  0.0 0.0  

stillinger_weber 6.589 2.0925 1.87 29.15 1.02 -0.303333333333 7.049556277 0.6822245584 3.2 0.4 0.0  

#### fit (1) and fix (0) parameters: 
#### Example:

1 1 1 1 1 1 0 1 1 1 0   

#### imposing the bounds on parameters  
#### format is: 
#### (index lower upper)  ... ): 
#### Example:

10 0 5 

    

### Step 4: run the optimization

* Run the following command either interactively or through a slurm script 

#### optimize -c 2 -j force_mathcing_tutorial -i input_file -Ref ../ReferenceData -prep ../prepsystem

* This may take 2 ~ 3 hours depending on the initial guess

### Step 5: results

#### The objective vs iterations: 
<img src="objective_vs_iteration.png" width="600">

#### potential energy distribution: 

<img src="energy_distribution.png" width="600">

#### forces distributions: 
   
<img src="guess_force.png" width="600"/>
<img src="best_force.png" width="600"/>



### Reference: 

[1]: Molinero, V., & Moore, E. B. (2009). Water Modeled As an Intermediate Element between Carbon and Silicon. J. Phys. Chem. B, 113(13), 4008–4016. https://doi.org/Doi 10.1021/Jp805227c \
[2]: Stillinger, F. H., & Rahman, A. (1974). Improved Simulation of Liquid Water by Molecular-Dynamics. J. Chem. Phys., 60(4), 1545–1557. https://doi.org/Doi 10.1063/1.1681229 \
[3]: Ercolessi, F., & Adams, J. B. (1994). Interatomic Potentials from First-Principles Calculations: The Force-Matching Method. Europhysics Letters ({EPL}), 26(8), 583–588. https://doi.org/10.1209/0295-5075/26/8/005
[4]: Wang, L. P., Chen, J., & Van Voorhis, T. (2013). Systematic parametrization of oolarizable force fields from quantum chemistry data. Journal of Chemical Theory and Computation. https://doi.org/10.1021/ct300826t